In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Reading the Data

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
df=pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/train.csv")

In [ ]:
df.head()

In [ ]:
# for i in range(1,100):
sns.kdeplot(df.loc[:,df.columns[1]])

# Checking for skeqness in data

In [ ]:
fig, axes=plt.subplots(10,10,figsize=(40,35))
j=0
i=0
for k in range(1,100):
    if j==10:
        i+=1
        j=0
    sns.kdeplot(df.loc[:,df.columns[k]],ax=axes[i,j])
    plt.gca().set_title(f"{df.columns[k]}")
    j+=1
    

In [ ]:
fig.savefig('myimage.svg', format='svg', dpi=1200)

# Looking for Outliers

In [ ]:
fig_out, axes_out=plt.subplots(10,10,figsize=(40,35))
j=0
i=0
for k in range(1,100):
    if j==10:
        i+=1
        j=0
    sns.boxplot(y=df.columns[k],x=df.columns[-1],data=df,ax=axes_out[i,j])
    #sns.kdeplot(df.loc[:,df.columns[k]],ax=axes[i,j])
    plt.gca().set_title(f"{df.columns[k]}")
    j+=1



In [ ]:
fig_out.savefig('outlier.svg', format='svg', dpi=1200)

# As the Data contains outliers, so using RobusScaler

In [ ]:
from sklearn.preprocessing import RobustScaler

In [ ]:
X_train=df.loc[:,df.columns[1:-1]].values
y_train=df.iloc[:,-1].values

In [ ]:
scaler=RobustScaler()
X_train=scaler.fit_transform(X_train)

# Preprocessing Data

## Creating new Data for AutoML to ML to work on

In [ ]:
df_train=pd.DataFrame(X_train)

In [ ]:
df_train.columns=df.columns[1:-1]

In [ ]:
df_train.reset_index(inplace=True)

In [ ]:
df_train.columns=df.columns[:-1]

In [ ]:
df_train.iloc[:,0]+=1

In [ ]:
df_train.set_index("id",inplace=True)

In [ ]:
df_train.head()

In [ ]:
df_train_full=pd.concat([df_train,pd.DataFrame(y_train)],axis=1)

In [ ]:
df_train_full.drop(0,inplace=True)

In [ ]:
df_train_full.columns=df.columns[1:]

In [ ]:
df_train_full.reset_index(inplace=True)

In [ ]:
df_train_full.columns=df.columns

In [ ]:
df_train_full.id-=1

In [ ]:
df_train_full.head()

In [ ]:
df_train_full.to_csv("new_train.csv",index=False)

In [ ]:
df_test=pd.read_csv("../input/tabular-playground-series-nov-2021/test.csv")

In [ ]:
df_test.head()

In [ ]:
df_test.shape

In [ ]:
df_test.columns[1:]

In [ ]:
X_test=df_test.loc[:,df_test.columns[1:]].values

In [ ]:
X_test[:,:].shape

In [ ]:
X_test=scaler.transform(X_test)

In [ ]:
test=pd.DataFrame(X_test)

In [ ]:
test.reset_index(inplace=True)

In [ ]:
test.head()

In [ ]:
test.columns=df_test.columns

In [ ]:
test.id+=600000

In [ ]:
test.head()

In [ ]:
test.head()

In [ ]:
test.to_csv("new_test.csv",index=False)

# Modeling using AutoML

In [ ]:
import h2o
print(h2o.__version__)
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='11G')

In [ ]:
train = h2o.import_file("./new_train.csv")


In [ ]:
test = h2o.import_file("./new_test.csv")

In [ ]:
train.head()

In [ ]:
print(f'Size of training set: {train.shape[0]} rows and {train.shape[1]} columns')

In [ ]:
x = train.columns
y = 'target'
x.remove(y)

In [ ]:
aml = H2OAutoML(max_runtime_secs = 3500*2, seed = 1, project_name = "lb_frame")
aml.train(x = x, y = y, training_frame = train)

In [ ]:
# import time 
# while True:
#     print("5")
#     time.sleep(60)


In [ ]:
lb = aml.leaderboard
lb.head()  

In [ ]:
aml.leader

In [ ]:
pred = aml.predict(test)
pred.head()

In [ ]:
pred.shape

In [ ]:
sample_submission = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')
sample_submission.shape

In [ ]:
sample_submission['target'] = pred.as_data_frame().values
sample_submission.to_csv('h2o_automl_submission_4.csv', index=False)